In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

2024-06-24 18:33:37.286979: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-24 18:33:37.287090: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-24 18:33:37.472765: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


### Load and Preprocess the data

In [2]:
#Define paths
train_dir='/kaggle/input/dogs-cats-small-dataset/train'
test_dir='/kaggle/input/dogs-cats-small-dataset/test1'

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#Create a image datagen
train_datagen=ImageDataGenerator(
rescale=1/255,
shear_range=0.2,
zoom_range=0.2,
horizontal_flip=True,
validation_split=0.2)

test_datagen=ImageDataGenerator(rescale=1/255)

#Load train and validation data
train_generator=train_datagen.flow_from_directory(
train_dir,
target_size=(128,128),
batch_size=32,
class_mode='binary',
subset='training') #Set as training data

validation_generator=train_datagen.flow_from_directory(
train_dir,
target_size=(128,128),
batch_size=32,
class_mode='binary',
subset='validation')

#Load test data
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary',
    shuffle=False)

Found 4800 images belonging to 2 classes.
Found 1200 images belonging to 2 classes.
Found 0 images belonging to 0 classes.


### Create and Train a CNN model

In [4]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.optimizers import Adam

# Define the CNN model
cnn_model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Compile the model
cnn_model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

# Display model summary to verify architecture
cnn_model.summary()

# Train the model
history_cnn = cnn_model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=10
)

/opt/conda/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │    12,845,568 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           513 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 12,939,329 (49.36 MB)

 Trainable params: 12,939,329 (49.36 MB)

...

150/150 ━━━━━━━━━━━━━━━━━━━━ 63s 347ms/step - accuracy: 0.5262 - loss: 0.7125 - val_accuracy: 0.5456 - val_loss: 0.6716
Epoch 2/10


/opt/conda/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


150/150 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.4375 - val_loss: 0.7376
Epoch 3/10


W0000 00:00:1719254100.672263      90 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


150/150 ━━━━━━━━━━━━━━━━━━━━ 31s 199ms/step - accuracy: 0.6276 - loss: 0.6544 - val_accuracy: 0.6495 - val_loss: 0.6331
Epoch 4/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 0s 599us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.6875 - val_loss: 0.6174
Epoch 5/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 32s 202ms/step - accuracy: 0.6619 - loss: 0.6291 - val_accuracy: 0.6959 - val_loss: 0.5996
Epoch 6/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 0s 578us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.6875 - val_loss: 0.6133
Epoch 7/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 32s 205ms/step - accuracy: 0.6916 - loss: 0.6046 - val_accuracy: 0.7280 - val_loss: 0.5450
Epoch 8/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 0s 626us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.7500 - val_loss: 0.5062
Epoch 9/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 32s 206ms/step - accuracy: 0.6958 - loss: 0.5830 - val_accuracy: 0.7188 - val_loss: 0.5623
Epoch 10/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 0s 624us/step - accuracy: 0.0000